# Proof of Concept - Read from MySQL Database

## Setup Steps

### Names
Go ahead and define what to name the parts.
```
export MYSQL_SERVER_CONTAINER="mysql-db"
export MYSQL_ROOT_PASSWORD="datascience" 
export DB_DOCKER_NETWORK="dbnet"
```

### Networking
Create a separate network for database containers, or just use the default docker bridge network, "bridge".
Containers must be in the same network to communicate.
```
docker network create --driver bridge $DB_DOCKER_NETWORK
```

### Database Server

```
docker run --detach --name=$MYSQL_SERVER_CONTAINER --net=$DB_DOCKER_NETWORK --env="MYSQL_ROOT_PASSWORD=$MYSQL_ROOT_PASSWORD" -p 6603:3306 mysql
```

This will use the [official mysql docker image](https://hub.docker.com/_/mysql/) to create a database server container with root password $MYSQL_ROOT_PASSWORD (defined above).
Capture the IP address of the DB server container with `docker inspect mysql-db | grep -i 'ipaddress'`
This will yeild something like
```
            "SecondaryIPAddresses": null,
            "IPAddress": "",
                    "IPAddress": "172.18.0.4",
```

### Command Line Client
To create a database and add data, access the mySql server through a command line by creating another mysql container linked to the server container. 
This container will only exist until you exit the mysql command prompt. 
Assume data you want to load into the database is in `/home/cownby/data/` on the host system.
(Note no shell variable substitution inside single quotes.)
```
docker run -it -v "/home/cownby/data/":"/data" --net=$DB_DOCKER_NETWORK --link "$MYSQL_SERVER_CONTAINER":mysql --rm mysql sh -c 'exec mysql -h172.18.0.4 -uroot -p'

```
You will be prompted for the password, $MYSQL_ROOT_PASSWORD

### Sample SQL Commands

```SQL
/*  
Assume volume is mounted to `/data` and data is in file `supers.csv`.

sample data:
id,firstName,lastName,superpower,goodDeeds,maxSpeed
0,100,carolyn,ownby,smile,72,15.5
0,200,krizia,conrad,good food!,80,25.7
0,300,jordan,dick,roadster,30,80.25
0,400,dixon,dick,vision,65,56.8

*/

create database giskard;
use giskard;
create table supers (
	key_id      int NOT NULL AUTO_INCREMENT UNIQUE,
	import_id   int,
	first_name  varchar(20),
	last_name   varchar(20),
	super_power varchar(20),
	good_deeds  int,
	max_speed   float
);

load data local infile '/data/supers.csv'
      into table supers
      fields terminated by ','
      lines terminated by '\n'
      ignore 1 rows;
```


## References

* [Excel to MySQL](http://www.prcconsulting.net/2016/10/migrating-an-excel-spreadsheet-to-mysql-and-to-spark-2-0-1-part-1/)  
This is NOT done programmatically, but by simply exporting an excel sheet as csv and importing into mySQL.
* [Spark & MySQL](http://www.prcconsulting.net/wp-content/uploads/2016/10/Connect_MySQL.py_-1.html)
* [Spark & database properties](http://spark.apache.org/docs/latest/sql-programming-guide.html#jdbc-to-other-databases)
* [tutorial](http://severalnines.com/blog/mysql-docker-containers-understanding-basics)
* [official mysql docker repository](https://hub.docker.com/_/mysql/)


##  Test Notes
Commands used (frequently!) to create the image & container for this notebook
```
export DB_NOTEBOOK_IMAGE="mysql-notebook-image"
export DB_NOTEBOOK_CONTAINER="mysql-notebook"

# create image for spark-mysql-python
time docker build -t $DB_NOTEBOOK_IMAGE . 1>build.log 2>build.err  ;ll
 
# create container for spark-mysql-python
docker run -d -p 8826:8888  -e GRANT_SUDO=yes --user root --net=$DB_DOCKER_NETWORK --name=$DB_NOTEBOOK_CONTAINER --link $MYSQL_SERVER_CONTAINER:$DB_NOTEBOOK_CONTAINER  $DB_NOTEBOOK_IMAGE  start-notebook.sh
 
# clean up spark-mysql-python image & container
 docker stop $DB_NOTEBOOK_CONTAINER ;docker rm $DB_NOTEBOOK_CONTAINER ;docker rmi $DB_NOTEBOOK_IMAGE
```

In [1]:
# Basic setup required in all notebooks
 
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf

# Define an arbitrary application name & stand-alone Spark cluster
appName='archetest'
master='local[*]' #local spark-master

# Explicitly define python 2 since we have both versions 2 & 3 installed
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

# Create spark context with which we will reference the Spark API
spark = (SparkSession
         .builder
         .master(master)
         .config(conf=SparkConf())  
         .appName(appName)
         .getOrCreate()
        )

spark.version


u'2.0.2'

In [3]:
# constants
MYSQL_DATABASE = 'giskard'
MYSQL_PASSWORD = 'ds'
MYSQL_USER='root'
DB_SERVER_IP='172.18.0.4'
TEST_TABLE='supers'

In [4]:
# verify can connect to database just through python
# no other installs required from base notebook container to access mysql via python,
# but the containers must be in the same network.
!sudo apt-get update && sudo apt-get install -y python-mysqldb

# from https://pypi.python.org/pypi/MySQL-python/1.2.5
import MySQLdb

db = MySQLdb.connect(host=DB_SERVER_IP,     # your host, usually localhost
                     user=MYSQL_USER,       # your username
                     passwd=MYSQL_PASSWORD, # your password
                     db=MYSQL_DATABASE)     # name of the data base


# you must create a Cursor object. It will let
#  you execute all the queries you need
cur = db.cursor()

# Use all the SQL you like
cur.execute("SELECT * FROM "+TEST_TABLE)

# print all the first cell of all the rows
for row in cur.fetchall():
    print row[2]

db.close()

Hit http://security.debian.org jessie/updates InRelease
Get:1 http://security.debian.org jessie/updates/main amd64 Packages [431 kB]
Ign http://httpredir.debian.org jessie InRelease
Get:2 http://httpredir.debian.org jessie-updates InRelease [145 kB]
Hit http://httpredir.debian.org jessie Release.gpg
Hit http://repos.mesosphere.io wheezy InRelease
Hit http://httpredir.debian.org jessie Release
Get:3 http://httpredir.debian.org jessie-updates/main amd64 Packages [17.6 kB]
Get:4 http://repos.mesosphere.io wheezy/main amd64 Packages [3,641 B]
Get:5 http://httpredir.debian.org jessie/main amd64 Packages [9,049 kB]
Fetched 9,646 kB in 6s (1,382 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-mysqldb is already the newest version.
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.
carolyn
krizia
jordan
dixon


In [5]:

URL = 'jdbc:mysql://{0}/{1}'.format(DB_SERVER_IP,MYSQL_DATABASE)

df = (spark.read.format('jdbc')
      .options(
        url=URL,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        dbtable=TEST_TABLE,
        driver='com.mysql.jdbc.Driver')
      .load()
     )

print df.head()
df.printSchema()

Row(key_id=4, import_id=100, first_name=u'carolyn', last_name=u'ownby', super_power=u'smile', good_deeds=72, max_speed=15.5)
root
 |-- key_id: integer (nullable = false)
 |-- import_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- super_power: string (nullable = true)
 |-- good_deeds: integer (nullable = true)
 |-- max_speed: double (nullable = true)



In [6]:
%env

{'APACHE_SPARK_VERSION': '2.0.2',
 'CLICOLOR': '1',
 'CONDA_DIR': '/opt/conda',
 'DEBIAN_FRONTEND': 'noninteractive',
 'GIT_PAGER': 'cat',
 'GRANT_SUDO': 'yes',
 'HOME': '/home/jovyan',
 'HOSTNAME': '5045900c3521',
 'JPY_PARENT_PID': '9',
 'LANG': 'en_US.UTF-8',
 'LANGUAGE': 'en_US.UTF-8',
 'LC_ALL': 'en_US.UTF-8',
 'LOGNAME': 'jovyan',
 'MAIL': '/var/mail/jovyan',
 'MESOS_NATIVE_LIBRARY': '/usr/local/lib/libmesos.so',
 'NB_UID': '1000',
 'NB_USER': 'jovyan',
 'PAGER': 'cat',
 'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
 'PWD': '/home/jovyan/work',
 'PYSPARK_PYTHON': '/opt/conda/envs/python2/bin/python',
 'PYTHONPATH': '/usr/local/spark/python:/usr/local/spark/python/lib/py4j-0.10.4-src.zip:/usr/lib/python2.7/dist-packages',
 'R_LIBS_USER': '/usr/local/spark/R/lib',
 'SHELL': '/bin/bash',
 'SHLVL': '1',
 'SPARK_HOME': '/usr/local/spark',
 'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog